# Explore here

In [1]:
# Your code here

import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")

data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [2]:
from sklearn.preprocessing import StandardScaler

data_types = data.dtypes
print(data_types)
numeric_columns = [c for c in list(data_types[data_types != "object"].index) if c != "Obesity_number"]

scaler = StandardScaler()
norm_features = scaler.fit_transform(data[numeric_columns])

# Create a new DataFrame with the scaled numerical variables
data_scal = pd.DataFrame(norm_features, index = data.index, columns = numeric_columns)
data_scal["Obesity_number"] = data["Obesity_number"]
data_scal.head()


fips                        int64
TOT_POP                     int64
0-9                         int64
0-9 y/o % of total pop    float64
19-Oct                      int64
                           ...   
CKD_prevalence            float64
CKD_Lower 95% CI          float64
CKD_Upper 95% CI          float64
CKD_number                  int64
Urban_rural_code            int64
Length: 108, dtype: object


,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,Obesity_number
0,-1.940874,-0.145679,-0.142421,0.158006,-0.135556,0.573496,-0.153144,0.027610,-0.139384,0.588469,...,-0.063696,-0.071720,-0.089834,-0.129902,-0.609615,-0.582796,-0.669652,-0.147523,-1.082865,15193
1,-1.940742,0.341296,0.287476,-0.242861,0.320383,-0.193107,0.183774,-0.469965,0.230620,-0.110300,...,-0.394103,-0.414900,-0.337677,0.376251,-0.433549,-0.393279,-0.343373,0.389791,-0.420704,50761
2,-1.940610,-0.237785,-0.239429,-0.419441,-0.246181,-0.439718,-0.225971,0.272104,-0.218759,0.656538,...,2.432709,2.483064,2.317776,-0.183415,1.855312,1.880929,1.777443,-0.204321,0.903618,8013
3,-1.940478,-0.245223,-0.246032,-0.426966,-0.254791,-0.609076,-0.230792,0.396168,-0.220555,1.264959,...,0.376846,0.423984,0.299632,-0.229096,-0.257483,-0.203761,-0.180233,-0.242100,-1.745026,6894
4,-1.940346,-0.138966,-0.135053,0.186249,-0.137140,0.216679,-0.155888,-0.200808,-0.143570,0.088582,...,0.156575,0.195197,0.158008,-0.111247,-0.081417,-0.014244,-0.017093,-0.124105,-1.745026,15112


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

X = data_scal.drop(columns=["Obesity_number"])
y = data_scal["Obesity_number"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
train_indices = list(X_train.index)
test_indices = list(X_test.index)

k = int(len(X_train.columns) * 0.3)
selection_model = SelectKBest(score_func = f_regression, k = k)
selection_model.fit(X_train, y_train)
ix = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns = X_train.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns = X_test.columns.values[ix])

X_train_sel.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Heart disease_number,COPD_number,diabetes_number,CKD_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.212643,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.225814,-0.223516,-0.218609,-0.219329
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.116680,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.115392,-0.110080,-0.131449,-0.130962
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.192263,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.216207,-0.193106,-0.189197,-0.206391
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,0.062458,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,-0.007276,-0.007077,-0.047515,-0.045054
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.274818,0.194913,0.097767,0.161314,0.088485,0.165555,0.197812,0.265603,0.123040,0.132454


In [4]:
X_train_sel["Obesity_number"] = list(y_train)
X_test_sel["Obesity_number"] = list(y_test)

X_train_sel.to_csv("../data/processed/clean_train.csv", index = False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index = False)

In [5]:

data = pd.concat([X_train_sel, X_test_sel])
data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Heart disease_number,COPD_number,diabetes_number,CKD_number,Obesity_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.225814,-0.223516,-0.218609,-0.219329,7930
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.115392,-0.110080,-0.131449,-0.130962,13713
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.216207,-0.193106,-0.189197,-0.206391,10238
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,-0.007276,-0.007077,-0.047515,-0.045054,25735
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.194913,0.097767,0.161314,0.088485,0.165555,0.197812,0.265603,0.123040,0.132454,38882


Modelo de Regresion Logistica

In [6]:


train_data = pd.read_csv("../data/processed/clean_train.csv")
test_data = pd.read_csv("../data/processed/clean_test.csv")

train_data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Heart disease_number,COPD_number,diabetes_number,CKD_number,Obesity_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.225814,-0.223516,-0.218609,-0.219329,7930
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.115392,-0.110080,-0.131449,-0.130962,13713
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.216207,-0.193106,-0.189197,-0.206391,10238
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,-0.007276,-0.007077,-0.047515,-0.045054,25735
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.194913,0.097767,0.161314,0.088485,0.165555,0.197812,0.265603,0.123040,0.132454,38882


In [7]:
X_train = train_data.drop(["Obesity_number"], axis = 1)
y_train = train_data["Obesity_number"]
X_test = test_data.drop(["Obesity_number"], axis = 1)
y_test = test_data["Obesity_number"]

In [8]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [9]:
print(f"Intercep (a): {model.intercept_}")
print(f"Coefficients: {model.coef_}")

Intercep (a): [-0.28200657 -0.28162106 -0.28019041 ... -2.35763505 -2.14656162
 -1.95450627]
Coefficients: [[-0.06898852 -0.06486313 -0.07092565 ... -0.11457999 -0.08005451
  -0.08051149]
 [-0.06884636 -0.06420885 -0.07028195 ... -0.11458041 -0.08018632
  -0.08084962]
 [-0.06868457 -0.06421204 -0.07029773 ... -0.11431065 -0.07991635
  -0.08053421]
 ...
 [ 0.2060507   0.1439869   0.10592765 ...  0.20873653  0.15620476
   0.28809476]
 [ 0.22413527  0.3823879   0.26440643 ...  0.04547226  0.52901934
   0.14791142]
 [ 0.20984838  0.10196576  0.11058003 ...  0.11086162  0.33236251
   0.24625605]]


In [10]:
y_pred = model.predict(X_test)
y_pred

array([  2523, 129387,   2523,  27506,  27506,   2523,   2523,   2523,
         2523,   2523,   2523,   2523,  10672,   2523,  27506,   2523,
       361693,   2523,   2523,   2523,   2523,   2523,   2523,   2523,
       231623,  10672,   2523,   2523,   2523,   2523,   2523,   2523,
         2523,  10672,  27506,   2523,  10672,   2523,   2523,  10672,
         2523,  84208,   2523,   2523,  10672,   2523,   2523,  27506,
         2523,   2523, 110189,   2523,   2523,   2523,  27506,   2523,
       225957,   2523,  10672,   2523,  27506,   2523,   2523,  27506,
         2523,  10672,   2523,   2523,   2523, 127859,  10672,   2523,
         2523,   2523,   2523,  10672, 110189,   2523,   2523,   2523,
         2523,  27506,  68286,   2523,   2523,   2523, 224533,   2523,
         2523,  55102,   2523,   2523,   2523,   2523,   2523,   2523,
         2523,   2523,  27506,   2523, 251006,  27506,   2523,   2523,
         2523,  10672,   8619,   2523,  27506, 104077,   2523,  27506,
      

In [20]:

from sklearn.metrics import mean_squared_error, r2_score

print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

MSE: 493459460.72292995
R2 Score: 0.7743244981160995
Coeficiente de determinación: 0.7743244981160995


Modelo Lasso (L1)

In [29]:
## Importar librerias
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score


In [30]:
## Busqueda de hiper-parámetros
lasso_m = {'alpha': np.logspace(-5, 3, 100)}

lasso_grid = GridSearchCV(Lasso(),
        lasso_m,
        n_jobs = 50,
        scoring= 'neg_root_mean_squared_error',
        cv= 5
       )

lasso_grid.fit(X_train, y_train)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.771e+09, tolerance: 1.389e+09
  model = cd_fast.enet_coordinate_descent(
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.122e+09, tolerance: 1.408e+09
  model = cd_fast.enet_coordinate_descent(
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

GridSearchCV(cv=5, estimator=Lasso(), n_jobs=50,
             param_grid={'alpha': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05, 7.74263683e-05,
       9.32603347e-05, 1.12332403e-04, 1.35304777e-04, 1.62975083e-04,
       1.96304065e-04, 2.36448941e-04, 2.84803...
       1.38488637e+01, 1.66810054e+01, 2.00923300e+01, 2.42012826e+01,
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03])},
             scoring='neg_root_mean_squared_error')

In [31]:
### Mejores parámetros
lasso_grid.best_params_

{'alpha': np.float64(61.35907273413176)}

In [32]:
## Mejor modelo
best_lasso_grid = Lasso(alpha = 61.35907273413176)
best_lasso_grid.fit(X_train, y_train)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.150e+09, tolerance: 1.602e+09
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=61.35907273413176)

In [33]:
## Presicción lasso
y_pred_lasso_grid = best_lasso_grid.predict(X_test)
y_pred_lasso_grid

array([2.36945756e+03, 6.37281379e+04, 6.70137666e+03, 2.80948541e+04,
       4.49582023e+04, 7.08188963e+03, 2.85991343e+03, 4.79690499e+03,
       1.24479907e+03, 3.35377587e+03, 5.48590567e+03, 5.67113118e+02,
       2.43949913e+04, 4.85944048e+03, 4.16876530e+04, 1.04859411e+04,
       2.62076031e+05, 4.74778907e+03, 8.90725354e+03, 1.43333377e+03,
       7.13116501e+03, 2.75132304e+03, 3.39682537e+03, 2.09422203e+03,
       5.39875612e+05, 1.24890011e+04, 3.08441697e+03, 8.13420601e+03,
       3.37490645e+03, 8.81083287e+02, 4.20032013e+03, 9.14299059e+03,
       9.56184958e+03, 1.21540966e+04, 3.46285128e+04, 6.25084866e+03,
       1.06488890e+04, 4.44496163e+03, 4.02793522e+03, 9.02018685e+03,
       5.91013964e+03, 5.57529870e+04, 1.32231763e+03, 4.04653469e+02,
       1.30421298e+04, 2.71488946e+03, 1.25578954e+03, 2.67132905e+04,
       3.44068236e+03, 2.55912932e+03, 5.68533393e+04, 8.73022621e+02,
       6.19004146e+03, 1.92760117e+03, 4.08806758e+04, 3.53886497e+03,
      

In [34]:
### Raiz cuadrada del error
print(f'MSE: {mean_squared_error(y_test, y_pred_lasso_grid)}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_pred_lasso_grid))}')
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred_lasso_grid)}")

MSE: 5880434.918778791
RMSE:2424.960807678918
Coeficiente de determinación: 0.9973106805984694


In [35]:
## Busqueda de hiper-parámetros empleando Random
lasso_m2 = {'alpha': np.linspace(0,50,400)}

lasso_ran = RandomizedSearchCV(Lasso(),
        lasso_m2,
        n_iter = 50,
        scoring= 'neg_root_mean_squared_error',
        cv = 5
       )

lasso_ran.fit(X_train, y_train)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.088e+10, tolerance: 1.389e+09
  model = cd_fast.enet_coordinate_descent(
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.122e+10, tolerance: 1.408e+09
  model = cd_fast.enet_coordinate_descent(
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

RandomizedSearchCV(cv=5, estimator=Lasso(), n_iter=50,
                   param_distributions={'alpha': array([ 0.        ,  0.12531328,  0.25062657,  0.37593985,  0.50125313,
        0.62656642,  0.7518797 ,  0.87719298,  1.00250627,  1.12781955,
        1.25313283,  1.37844612,  1.5037594 ,  1.62907268,  1.75438596,
        1.87969925,  2.00501253,  2.13032581,  2.2556391 ,  2.38095238,
        2.50626566,  2.63157895,  2.75689223,  2.88220551,  3.0075188...
       46.36591479, 46.49122807, 46.61654135, 46.74185464, 46.86716792,
       46.9924812 , 47.11779449, 47.24310777, 47.36842105, 47.49373434,
       47.61904762, 47.7443609 , 47.86967419, 47.99498747, 48.12030075,
       48.24561404, 48.37092732, 48.4962406 , 48.62155388, 48.74686717,
       48.87218045, 48.99749373, 49.12280702, 49.2481203 , 49.37343358,
       49.49874687, 49.62406015, 49.74937343, 49.87468672, 50.        ])},
                   scoring='neg_root_mean_squared_error')

In [36]:
### Mejores parámetros
lasso_ran.best_params_

{'alpha': np.float64(49.87468671679198)}

In [37]:
## Mejor modelo
best_lasso = Lasso(alpha = 49.87468671679198)
best_lasso.fit(X_train, y_train)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.664e+09, tolerance: 1.602e+09
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=49.87468671679198)

In [38]:
## Presicción lasso
y_pred_lasso = best_lasso.predict(X_test)
y_pred_lasso

array([ 2.34123361e+03,  6.25655414e+04,  6.66871300e+03,  2.90325599e+04,
        4.53551298e+04,  7.07435644e+03,  2.89726616e+03,  4.79431251e+03,
        1.23313836e+03,  3.17339174e+03,  5.57162061e+03,  5.10377110e+02,
        2.44989661e+04,  4.90537758e+03,  4.13755898e+04,  1.05028561e+04,
        2.60754204e+05,  4.73071977e+03,  8.87478338e+03,  1.41324703e+03,
        7.01701190e+03,  2.78730400e+03,  3.46841247e+03,  2.05856517e+03,
        5.42447578e+05,  1.24610180e+04,  3.08000850e+03,  8.39924833e+03,
        3.46536411e+03,  8.35508980e+02,  4.17194769e+03,  9.08459246e+03,
        9.64254941e+03,  1.26282367e+04,  3.44277840e+04,  6.30607623e+03,
        1.06296421e+04,  4.57521750e+03,  4.03044579e+03,  9.24909515e+03,
        5.73794467e+03,  5.43813241e+04,  1.30487439e+03,  3.49860641e+02,
        1.31762160e+04,  2.72271183e+03,  1.22379693e+03,  2.69508850e+04,
        3.44409943e+03,  2.53811546e+03,  5.67716476e+04,  8.10324074e+02,
        6.13001307e+03,  

In [39]:
### Raiz cuadrada del error
print(f'MSE: {mean_squared_error(y_test, y_pred_lasso)}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_pred_lasso))}')
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred_lasso)}")

MSE: 6647670.3988040285
RMSE:2578.307661782051
Coeficiente de determinación: 0.9969597981738744


In [43]:
print(f"Intercepto (a) grid: {best_lasso_grid.intercept_}")
# print(f"Coeficientes (b1, b2) grid: {best_lasso_grid.coef_}")

# print("----------------------------------------------------------------------")

print(f"Intercepto (a) random: {best_lasso.intercept_}")
# print(f"Coeficientes (b1, b2) random: {best_lasso.coef_}")

Intercepto (a) grid: 25262.22356301866
Intercepto (a) random: 25257.66521743368


Modelo Ridge (L2)

In [44]:
## Busqueda de hiper-parámetros
ridge_m = {'alpha': np.linspace(0,10,200)}

ridge_grid = GridSearchCV(Ridge(),
        ridge_m,
        n_jobs = 50,
        scoring= 'neg_root_mean_squared_error',
        cv= 5
       )

ridge_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(), n_jobs=50,
             param_grid={'alpha': array([ 0.        ,  0.05025126,  0.10050251,  0.15075377,  0.20100503,
        0.25125628,  0.30150754,  0.35175879,  0.40201005,  0.45226131,
        0.50251256,  0.55276382,  0.60301508,  0.65326633,  0.70351759,
        0.75376884,  0.8040201 ,  0.85427136,  0.90452261,  0.95477387,
        1.00502513,  1.05527638,  1.10552764,  1.15577889,  1.20603015,
        1.25628141,...
        8.54271357,  8.59296482,  8.64321608,  8.69346734,  8.74371859,
        8.79396985,  8.84422111,  8.89447236,  8.94472362,  8.99497487,
        9.04522613,  9.09547739,  9.14572864,  9.1959799 ,  9.24623116,
        9.29648241,  9.34673367,  9.39698492,  9.44723618,  9.49748744,
        9.54773869,  9.59798995,  9.64824121,  9.69849246,  9.74874372,
        9.79899497,  9.84924623,  9.89949749,  9.94974874, 10.        ])},
             scoring='neg_root_mean_squared_error')

In [45]:
### Mejores parámetros
ridge_grid.best_params_

{'alpha': np.float64(0.45226130653266333)}

In [46]:
## Mejor modelo
best_ridge_grid = Ridge(alpha = 0.45226130653266333)
best_ridge_grid.fit(X_train, y_train)

Ridge(alpha=0.45226130653266333)

In [47]:
## Presicción ridge
y_pred_ridge_grid = best_ridge_grid.predict(X_test)
y_pred_ridge_grid

array([2.49622981e+03, 6.21957077e+04, 6.31911677e+03, 2.93735474e+04,
       4.42043853e+04, 6.75986753e+03, 3.18146509e+03, 4.55349102e+03,
       1.25530310e+03, 3.85544127e+03, 5.71336122e+03, 5.71487749e+02,
       2.49129646e+04, 4.87812546e+03, 4.17465198e+04, 1.07945694e+04,
       2.63904464e+05, 4.92213550e+03, 8.65440155e+03, 1.42853447e+03,
       6.75877969e+03, 2.90420474e+03, 3.52263522e+03, 2.26822684e+03,
       5.53778306e+05, 1.31007729e+04, 3.05874663e+03, 9.00104422e+03,
       3.67019622e+03, 8.70884589e+02, 4.32940644e+03, 8.64837830e+03,
       9.80335432e+03, 1.30164924e+04, 3.55440563e+04, 6.41602251e+03,
       1.05182241e+04, 4.86220494e+03, 3.93358832e+03, 9.37733547e+03,
       5.53303935e+03, 5.33080481e+04, 1.39036626e+03, 4.37953680e+02,
       1.33006721e+04, 2.65983377e+03, 1.25531193e+03, 2.46764042e+04,
       3.60775872e+03, 2.53670524e+03, 5.52437028e+04, 9.73320312e+02,
       6.01002405e+03, 2.04664253e+03, 4.20850516e+04, 3.41500058e+03,
      

In [48]:
### Raiz cuadrada del error
print(f'MSE: {mean_squared_error(y_test, y_pred_ridge_grid)}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_pred_ridge_grid))}')
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred_ridge_grid)}")

MSE: 5798692.484714329
RMSE:2408.0474423719998
Coeficiente de determinación: 0.9973480641452468


In [49]:
## Busqueda de hiper-parámetros empleando Random
ridge_m2 = {'alpha': np.linspace(0,50,400)}

ridge_ran = RandomizedSearchCV(Ridge(),
        ridge_m2,
        n_iter = 50,
        scoring= 'neg_root_mean_squared_error',
        cv = 5
       )

ridge_ran.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=Ridge(), n_iter=50,
                   param_distributions={'alpha': array([ 0.        ,  0.12531328,  0.25062657,  0.37593985,  0.50125313,
        0.62656642,  0.7518797 ,  0.87719298,  1.00250627,  1.12781955,
        1.25313283,  1.37844612,  1.5037594 ,  1.62907268,  1.75438596,
        1.87969925,  2.00501253,  2.13032581,  2.2556391 ,  2.38095238,
        2.50626566,  2.63157895,  2.75689223,  2.88220551,  3.0075188...
       46.36591479, 46.49122807, 46.61654135, 46.74185464, 46.86716792,
       46.9924812 , 47.11779449, 47.24310777, 47.36842105, 47.49373434,
       47.61904762, 47.7443609 , 47.86967419, 47.99498747, 48.12030075,
       48.24561404, 48.37092732, 48.4962406 , 48.62155388, 48.74686717,
       48.87218045, 48.99749373, 49.12280702, 49.2481203 , 49.37343358,
       49.49874687, 49.62406015, 49.74937343, 49.87468672, 50.        ])},
                   scoring='neg_root_mean_squared_error')

In [50]:
### Mejores parámetros
ridge_ran.best_params_

{'alpha': np.float64(1.6290726817042605)}

In [52]:
## Mejor modelo
best_ridge = Ridge(alpha = 1.6290726817042605)
best_ridge.fit(X_train, y_train)

Ridge(alpha=1.6290726817042605)

In [53]:
## Presicción ridge
y_pred_ridge = best_ridge.predict(X_test)
y_pred_ridge

array([2.39394985e+03, 6.15656826e+04, 6.40721260e+03, 2.99414821e+04,
       4.54951312e+04, 6.75890801e+03, 3.06866092e+03, 4.59260265e+03,
       1.21811676e+03, 3.63202849e+03, 5.61741269e+03, 5.26175131e+02,
       2.51574043e+04, 4.77247906e+03, 4.17628516e+04, 1.06811035e+04,
       2.61110403e+05, 4.84374120e+03, 8.58523247e+03, 1.43630171e+03,
       6.71164013e+03, 2.85711653e+03, 3.53726452e+03, 2.13143735e+03,
       5.47868156e+05, 1.30389672e+04, 3.03374910e+03, 9.00271931e+03,
       3.60692940e+03, 8.40468903e+02, 4.25940510e+03, 8.55218901e+03,
       9.78821692e+03, 1.30826376e+04, 3.54784475e+04, 6.41943674e+03,
       1.06762911e+04, 4.81717985e+03, 3.85179153e+03, 9.41499992e+03,
       5.46982790e+03, 5.33847172e+04, 1.33182048e+03, 3.70816494e+02,
       1.34874577e+04, 2.68069102e+03, 1.21805789e+03, 2.56158435e+04,
       3.53709710e+03, 2.55393044e+03, 5.53887657e+04, 9.03114892e+02,
       6.16144506e+03, 1.97469867e+03, 4.16302302e+04, 3.31119689e+03,
      

In [54]:
### Raiz cuadrada del error
print(f'MSE: {mean_squared_error(y_test, y_pred_ridge)}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_pred_ridge))}')
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred_ridge)}")

MSE: 6630400.254782272
RMSE:2574.9563597820975
Coeficiente de determinación: 0.9969676963878715


In [56]:
print(f"Intercepto (a) grid: {best_ridge_grid.intercept_}")
# print(f"Coeficientes (b1, b2) grid: {best_ridge_grid.coef_}")

print("----------------------------------------------------------------------")

print(f"Intercepto (a) random: {best_ridge.intercept_}")
# print(f"Coeficientes (b1, b2) random: {best_ridge.coef_}")

Intercepto (a) grid: 25265.774155945794
----------------------------------------------------------------------
Intercepto (a) random: 25257.569826557537
